## Methodology: Steps for Five RAG-Based Summarization Methods

#### Method 1: Normal RAG
- **Objective**: Establish a baseline summary using simple RAG.
- **Steps**:
  1. Input: `content`.
  2. Split into chunks (chunk_size=300, chunk_overlap=100) using RecursiveCharacterTextSplitter.
  3. Generate embeddings with SentenceTransformer (`nomic-ai/nomic-embed-text-v1`).
  4. Index embeddings in Qdrant (`content_collection`).
  5. Retrieve chunks with `retrieve` (query: "Summarize the content...", k=12).
  6. Build prompt: "Summarize:\nContent: {context}\nCreate a 280+ word summary with structure, formal tone, and coherence."
  7. Generate summary with `mistral-saba-24b`.

#### Method 2: Full-Featured (Small & Large Models + Analysis + Retrieval)
- **Objective**: Enhance summary with analysis and retrieval.
- **Steps**:
  1. Input: `content`.
  2. Extract keywords with `qwen-2.5-32b` using `extract_keywords_chunked`.
  3. Perform stylistic analysis with `qwen-2.5-32b` via `style_analysis_prompt`.
  4. Retrieve chunks with `retrieve`.
  5. Construct `summarization_prompt` with context, stylistic analysis, and keywords.
  6. Generate summary with `mistral-saba-24b`.

#### Method 3: Full-Featured with Vector Database Reset (Keep Analysis and Retrieval)
- **Objective**: Enhance summary with database reset while retaining analysis and retrieval.
- **Steps**:
  1. Input: `content`.
  2. Reuse keyword and stylistic analysis from Method 2.
  3. Reset Qdrant database.
  4. Re-index embeddings in Qdrant.
  5. Retrieve chunks with `retrieve`.
  6. Construct `summarization_prompt` with original analysis, new context.
  7. Generate summary with `mistral-saba-24b`.

#### Method 4: Full-Featured with Analysis and Retrieval Reset (Keep Database)
- **Objective**: Enhance summary with reset analysis and retrieval, maintaining the database.
- **Steps**:
  1. Input: `content`.
  2. Reset and re-extract keywords with adjusted `keyword_extraction_prompt` using `qwen-2.5-32b`.
  3. Reset and re-analyze style with modified `style_analysis_prompt`.
  4. Use existing Qdrant database from Method 3.
  5. Retrieve chunks with `retrieve` (query: "Summarize with emphasis on innovations").
  6. Construct `summarization_prompt` with new analysis and context.
  7. Generate summary with `mistral-saba-24b`.

#### Method 5: Full-Featured with Full Reset (Analysis, Retrieval, and Database)
- **Objective**: Reset all components to explore their combined impact.
- **Steps**:
  1. Input: `content`.
  2. Reset and re-extract keywords (focus on technical terms) with `qwen-2.5-32b`.
  3. Reset and re-analyze style (focus on readability).
  4. Reset Qdrant database.
  5. Re-index embeddings in Qdrant.
  6. Retrieve chunks with `retrieve` (query: "Summarize with emphasis on challenges").
  7. Construct `summarization_prompt` with new analysis and context.
  8. Generate summary with `mistral-saba-24b`.

---

### Notes
- **Evaluation**: Assess summaries using ROUGE-1, ROUGE-L, and BLEU metrics post-each method.
- **Script Adjustments**: Ensure `RecursiveCharacterTextSplitter` uses chunk_size=300, chunk_overlap=100, add_start_index=True; `retrieve` uses k=12; add Qdrant persistence for Method 4 if required.
- **Paper Use**: Include in the methodology section, followed by experimental setup and results comparing summary quality.

In [ ]:
#@title Qroq model and API Cohere and PDF
#Step 1:
%%capture
!pip install cohere langchain-cohere groq langchain langchain-community pypdf langchain-groq tiktoken txtai qdrant-client sentence-transformers
!pip install Qdrant

In [ ]:
#@title libraries //////////////////////////////////////////////////////////////////////////////////////
import os
import time
import csv
from groq import Groq
from langchain_core.documents import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from qdrant_client import QdrantClient
from qdrant_client.http import models
from sentence_transformers import SentenceTransformer

In [ ]:
#@title content 1
# Content to summarize (can be modified in a separate cell)
content = "Increasing Notch signaling antagonizes PRC2-mediated silencing to promote reprograming of germ cells into neurons Cell-fate decisions are controlled , on the one hand , by intercellular signaling and , on the other hand , by intrinsic mechanisms such as epigenetic chromatin modifications . The Notch signaling pathway is a highly conserved and widespread signaling mechanism ( Artavanis-Tsakonas et al . , 1999; Greenwald and Kovall , 2013 ) , which has been implicated in key cell-fate decisions such as the decision between proliferation and differentiation ( Liu et al . , 2010 ) . Notch signaling has also been implicated in cellular reprograming . Upon inhibition of Notch signaling , the oncogenic genes KLF4 and cMyc become dispensable for the generation of induced pluripotent stem cells ( iPSCs ) from mouse and human keratinocytes ( Ichida et al . , 2014 ) . In this setting , Notch inhibits reprograming . Conversely , Notch signaling promotes transdifferentiation of pancreatic acinar cells to ductal cells ( Sawey et al . , 2007 ) , or the conversion of hepatocytes into biliary cells in liver primary malignancy intrahepatic cholangiocarcinoma ( ICC ) ( Sekiya and Suzuki , 2012 ) . Notch signaling can also affect reprograming in normal development . In C . elegans , signaling through the GLP-1 and LIN-12 Notch receptors impedes reprograming during embryogenesis and , during larval development , signaling through LIN-12 is required for the conversion of a rectal epithelial cell into a motorneuron ( Jarriault et al . , 2008; Djabrayan et al . , 2012 ) . The role of epigenetic regulators in cell-fate decisions has been studied mostly in pluripotent cells cultured outside of their normal tissue environment ( Meshorer and Misteli , 2006; Spivakov and Fisher , 2007; Lessard and Crabtree , 2010; Orkin and Hochedlinger , 2011 ) . Therefore , the\\u00a0epigenetic regulation of stem cell identity in intact tissues remains poorly understood . Additionally , the impact of external cues , for example signaling from a stem cell niche to the recipient cell\\u2019s chromatin remains equally unresolved . By contrast , C . elegans has been used as a model to study reprograming in an intact organism ( Horner et al . , 1998; Fukushige et al . , 1998; Zhu et al . , 1998; Fukushige and Krause , 2005; Ciosk et al . , 2006; Jarriault et al . , 2008; Yuzyuk et al . , 2009; Riddle et al . , 2013 ) . In this model , germ cells can be directly reprogrammed into neurons by depleting specific chromatin modifiers such as LIN-53 ( Rbbp4/7 ) or components of PRC2 , and by concomitant overexpression of the transcription factor CHE-1 , which induces glutamatergic neuronal fate in a process which we refer to as Germ cell Conversion ( GeCo ) ( Tursun et al . , 2011; Patel et al . , 2012 ) . Here , we identify the Notch signaling pathway as a critical player in this reprograming model . This was unanticipated , since signaling through the Notch receptor GLP-1 ( henceforth GLP-1Notch ) from the somatic gonadal niche is known to maintain germline stem cell/progenitor fate ( Kimble and Crittenden , 2007 ) . To understand this novel , reprograming-promoting role of GLP-1Notch , we combined genetics with tissue-specific expression profiling . We identified genes regulated by GLP-1Notch , including genes recently shown to maintain the germline stem/progenitor cells ( Kershner et al . , 2014 ) . Additionally , and unexpectedly , we found that many genes activated by GLP-1Notch signaling were also repressed by the cell fate-stabilizing chromatin regulator PRC2 . We show that GLP-1Notch and PRC2 have an antagonistic effect on germ cell-fate decisions and demonstrate co-regulation of their common target , utx-1 . Importantly , UTX-1 is a histone demethylase known to erase the gene-silencing methylation of histone H3 dependent on PRC2 ( Maures et al . , 2011; Jin et al . , 2011; Vandamme et al . , 2012 ) . Thus , we propose that the GLP-1Notch\\u2013dependent induction of UTX-1 facilitates reprograming by alleviating PRC2-mediated repression of alternative cell fates . Germ cells can be converted into neuronal cells in intact C . elegans upon overexpression of the neuronal transcription factor CHE-1 , simply by depleting the chromatin modifier LIN-53 ( Tursun et al . , 2011; Patel et al . , 2012 ) . This GeCo phenotype can be followed in living animals by monitoring a reporter GFP expressed from the gcy-5 promoter , which otherwise is induced in glutamatergic ASE neurons ( Altun-Gultekin et al . , 2001 ) . In contrast to the spontaneous teratomatous differentiation of meiotic germ cells , observed in the absence of specific RNA-binding proteins ( Ciosk et al . , 2006; Biedermann et al . , 2009; Tocchini et al . , 2014 ) , GeCo is preferentially observed in the pre-meiotic , proliferating germ cells ( Tursun et al . , 2011; Patel et al . , 2012 ) . Consistently , removing the proliferating germ cells , by inhibiting GLP-1Notch signaling , prevents GeCo ( Tursun et al . , 2011 ) . However , because the proliferating germ cells were eliminated , these experiments did not address a possible direct effect of GLP-1Notch signaling on GeCo . We began addressing this issue by examining the gonads of animals carrying the gain-of-function glp-1 allele ( ar202 ) ( Pepper et al . , 2003 ) . These gonads are filled with proliferating germ cells and , upon depleting LIN-53 and overexpressing CHE-1 , we observed that significantly more germ cells converted to ASE neurons ( Figure 1A , Figure 1\\u2014source data 1 ) . We refer to this enhanced GeCo as 'GeCo+' . Detailed quantification revealed that the GeCo+ gonads contained more than twice the number of converted cells ( Figure 1\\u2014figure supplement 1A , Figure 1\\u2014source data 1 ) . The nuclei of these converted cells were reminiscent of neuronal nuclei and the cells displayed axo-dendritic projection ( Figure 1\\u2014figure supplement 1B ) , as previously described ( Tursun et al . , 2011; Patel et al . , 2012 ) . To confirm that the GeCo enhancement depends on the canonical Notch signaling pathway , rather than an independent function of the GLP-1Notch\\u00a0receptor , we RNAi-depleted the transcriptional effector of GLP-1Notch signaling , LAG-1 ( Christensen et al . , 1996 ) . We exposed animals only after hatching to lag-1 RNAi in order to avoid sterility , which is caused when animals are subjected to lag-1 RNAi earlier ( Supplemental file 1 ) . RNAi-mediated knock-down of lag-1 strongly inhibited GeCo ( Figure 1B , Figure 1\\u2014source data 1 ) . Importantly , under these experimental conditions , we did not observe any obvious reduction of germ cell numbers ( Figure 1C , Figure 1\\u2014source data 1 ) , suggesting a proliferation-independent effect of GLP-1Notch signaling on cell-fate conversion . To investigate this further , we tested GeCo efficiency on germ cells proliferating independently of GLP-1Notch signaling . We took advantage of mutants in which , in the absence of two meiosis/differentiation-promoting factors GLD-1 and GLD-2 , germ cells proliferate independently of GLP-1Notch ( Kadyk and Kimble , 1998 ) . Specifically , we examined GeCo in the loss-of-function gld-1 ( q497 ) gld-2 ( q485 ) mutants , which carried either wild-type glp-1 or the loss-of-function glp-1 ( q175 ) allele ( Austin and Kimble , 1987 ) . Both mutant combinations have previously been described to have tumorous germlines and impaired meiotic entry ( Kadyk and Kimble , 1998; Hansen et al . , 2004 ) . In contrast to efficient GeCo observed in the gld-1 ( q497 ) gld-2 ( q485 ) gonads , GeCo was strongly diminished in the gld-1 ( q497 ) gld-2 ( q485 ) ; glp-1 ( q175 ) gonads , despite the ongoing germ cell proliferation ( Figure 1D , Figure 1\\u2014source data 1 ) . Counting the number of germ cells in these gonads revealed only a slight difference ( a 15% increase in the numbers in the double vs . triple mutant gonads ) , suggesting that the strong enhancement of GeCo by GLP-1Notch signaling cannot be explained by the\\u00a0increased number of germ cells ( Figure 1\\u2014figure supplement 2 , Figure 1\\u2014source data 1 ) . Since it has been proposed that dividing cells have a higher propensity for cellular reprograming ( Egli et al . , 2008; Hanna et al . , 2009 ) , we also tested whether blocking the cell cycle would affect the observed GeCo enhancement in glp-1 ( gf ) gonads . As previously described ( Fox et al . , 2011; Patel et al . , 2012 ) , we used hydroxyurea ( HU ) treatment to block the cell cycle in the S phase . Blocking the cell cycle by HU did not diminish the GeCo+ phenotype ( Figure 1\\u2014figure supplement 3 , Figure 1\\u2014source data 1 ) . Combined , these results suggest that GLP-1Notch enhances GeCo independently from its role in promoting germ cell proliferation . 10 . 7554/eLife . 15477 . 003Figure 1 . GLP-1Notch signaling promotes reprograming of germ cells . ( A ) GLP-1Notch enhances germ cell conversion ( GeCo ) into neuronal-like cells . Left: Fluorescent ( top ) and combined fluorescent/differential interference contrast ( DIC ) micrographs ( bottom ) of adult animals . All animals ectopically expressed the pro-neuronal transcription factor CHE-1 from a heat-shock promoter . glp-1 ( ar202 ) is a temperature-sensitive gain-of-function allele of the Notch receptor . Animals were subjected to either mock ( control ) or lin-53 RNAi . Reprogrammed cells expressed a GFP reporter driven from the neuronal gcy-5 promoter ( here an in other figures nGFP ) and are outlined here and elsewhere in yellow . Any signal outside the outlined region comes from somatic tissues . GeCo+ indicates animals that displayed a strongly enhanced GeCo phenotype . Scale bars = 10 \\u03bcm . The cartoons depicting the GeCo and GeCo+ phenotypes are on the top right . The gonads are shaded in grey and GFP-positive converted germ cells are green . Fractions of animals displaying GeCo and GeCo+ are indicated below . At least 250 animals were quantified per condition . P-values were calculated using Student's t-test: p1<0 , 0001; p2=0 , 0006 . Error bars represent SEM . ( B ) The transcriptional effector of the GLP-1Notch signaling pathway , LAG-1 , is required for the GLP-1Notch\\u2013mediated enhancement of GeCo . Left: Fluorescent micrographs of adults expressing CHE-1\\u2013induced nGFP as explained above . GeCo is diminished upon the depletion of LAG-1 . White dashed lines outline the animal body . Scale bars = 10 \\u03bcm . Right: The corresponding quantifications . At least 400 animals were quantified per condition . P-values were calculated using Student's t-test: p1<0 , 0001; p2=0 , 0018 . Error bars represent SEM . ( C ) GLP-1Notch signaling enhances GeCo independently from germ cell proliferation . Shown are DAPI-stained gonads of glp-1 ( ar202 ) animals , expressing CHE-1\\u2013induced nGFP , treated with either mock or lin-53 RNAi . Germ cells were counted from the DTC ( yellow asterisk ) to the turn of the gonad arm ( dashed yellow line ) . 15 gonad arms per condition were counted . Scale bars = 10 \\u03bcm . Quantifications are on the right . While greatly inhibiting GeCo , lag-1 RNAi did not change the number of germ cells . P-values were calculated using Student's t-test: p1=0 , 89 . Error bars represent SEM . ( D ) GLP-1Notch enhances GeCo independently from proliferation . Left: Fluorescent micrographs of adults ( with indicated genotypes ) , expressing CHE-1\\u2013induced nGFP . The first panel on the left shows a control , heterozygous ( wild-type ) gld-1 gld-2/++; glp-1/+ animal . The other panels show the homozygous gld-1 ( q497 ) gld-2 ( q485 ) mutants , carrying either a loss-of-function ( q175 , center ) or a wild-type ( right ) allele of glp-1 . Despite proliferating , germ cells in the gld-1 gld-2; glp-1 gonads have lost the ability to undergo GeCo . Scale bars = 10 \\u03bcm . Right: the corresponding quantifications . At least 250 animals were quantified per condition . P-values were calculated using Student's t-test: p1=0 , 0478; p2=0 , 0201 . Error bars represent SEM . DOI: http://dx . doi . org/10 . 7554/eLife . 15477 . 00310 . 7554/eLife . 15477 . 004Figure 1\\u2014source data 1 . Quantification of GeCo in glp-1 ( gf ) and lag-1 RNAi\\u00a0animals . ( A ) Quantification of GeCo+ phenotype upon RNAi against lin-53 in glp-1 ( ar202 ) mutants . ( B ) Quantification of GeCo dependency on LAG-1 . ( C ) Quantification of germ cells in glp-1 ( ar202 ) gonads with our without lag-1 RNAi treatment . ( D ) Quantification of GeCo in different genetic backgrounds with highly proliferative germlines upon RNAi against lin-53 . Figure 1\\u2014figure supplement 1A source data: Quantification of gfp-positive germ cells . Figure 1\\u2014figure supplement 2 source data: Quantification of germ cells in different genetic backgrounds with highly proliferative germlines . Figure 1\\u2014figure supplement 3A source data: Quantification of GeCo after cell cycle block with HU treatment . More details can be found in the corresponding figure legends . DOI: http://dx . doi . org/10 . 7554/eLife . 15477 . 00410 . 7554/eLife . 15477 . 005Figure 1\\u2014figure supplement 1 . glp-1 ( gf ) gonads contain more than twice the number of converted cells which display neuronal characteristics . ( A ) For the quantification of gcy-5::gfp-positive cells per gonadal arm only the GeCo category of wt vs . glp-1 ( gf ) was used because GeCo+ animals already show an extensive area of the gonad filled with gcy-5::gfp-positive cells with usually >100 cells/gonad making reliable counting impossible . Notably , animals with a seemingly similar extend of GeCo in wt vs . glp-1 ( gf ) show a clear increase of gcy-5::gfp-positive cells per gonadal arm from approx . 10 in wt to > 30 in glp-1 ( gf ) . n ( wt ) = 75 gonaldal arms , n ( glp-1 ( ar202 ) ) = 221 gonadal arms . The background of the loss of function allele glp-1 ( q175 ) leads to a significant decrease in GeCo as shown previously ( Tursun et al . , 2011 ) . ( B ) A magnified view of gcy-5::gfp-positive ( nGFP ) cells , in a GeCo+ gonad from a glp-1 ( gf ) animal . The converted cells show axo-dendritic projections ( white arrow heads ) . The inset in the corresponding DIC image , magnified in the right-bottom corner , shows the nuclear morphology of a converted germ cell , which has lost the germ cell-specific \\u2018fried-egg\\u2019-like shape and instead shows nuclear speckles characteristic of a neuronal cell . Scale bar = 1 \\u03bcm . DOI: http://dx . doi . org/10 . 7554/eLife . 15477 . 00510 . 7554/eLife . 15477 . 006Figure 1\\u2014figure supplement 2 . Germ cell numbers are similar between gld-1 gld-2 double and gld-1 gld-2; glp-1 triple mutants . DAPI-stained gonads of gld-1 ( q497 ) gld-2 ( q485 ) or gld-1 ( q497 ) gld-2 ( q485 ) ; glp-1 ( q175 ) mutants , carrying the hsp::che-1 and gcy-5::gfp transgenes . The gonads were imaged by fluorescent microscopy using Z-stack acquisitions . Germ cells from the DTCs ( yellow asterisks ) to the turn of the gonad arm ( dashed lines ) were counted . Below:15 gonad arms per condition of L4 animals were counted . The numbers of germ cells differ only slightly ( 15% ) in the double mutant vs . triple mutants background . Scale bars = 10 \\u03bcm . Error bars represent SEM . DOI: http://dx . doi . org/10 . 7554/eLife . 15477 . 00610 . 7554/eLife . 15477 . 007Figure 1\\u2014figure supplement 3 . Blocking the cell cycle with hydroxyurea does not inhibit GeCo+ . ( A ) We used hydroxyurea ( HU ) treatment for 5 hr to chemically block the cell cycle , which makes germ cells arrest in the S phase of the cell cycle . This arrest does not diminish the GeCo+ phenotype in glp-1 ( gf ) gonads upon lin-53 RNAi and che-1 ( oe ) . At least 150 animals were quantified per condition . P-values were calculated using Student's t-test: p1=0 , 1409; p2=0 , 4583 . Error bars represent SEM . The right panel shows examples of GeCo+ displaying animals based on gcy-5::gfp ( nGFP ) for HU-untreated ( -HU ) and HU-treated ( +HU ) animals . ( B ) The gonads were stained for EdU incorporation . Dashed lines outline gonads . Asterisks indicate distal tips of gonads . Scale bars = 10 \\u03bcm . ( C ) The gonads of glp-1 ( ar202 ) gf animals , which were treated with HU for 12 hr and stained with DAPI and H3Ser10ph ( pH3 ) antibody . The pH3-positive cells indicate condensed chromosomes of dividing cells . After 12 hr of HU treatment , the gonads contained , as expected , enlargement nuclei ( arrowheads ) ( Gartner et al . , 2004; Fox et al . , 2011 ) . The loss of pH3-positive cells indicates a cell cycle arrest . Asterisks indicate distal tips of gonads , dashed lines outline gonad . Scale bars = 10 \\u03bcm . DOI: http://dx . doi . org/10 . 7554/eLife . 15477 . 007 To understand the effects of GLP-1Notch on GeCo , we set out to identify genes regulated by GLP-1Notch signaling in germ cells . To conduct the\\u00a0analysis in morphologically similar tissue , we again took advantage of the gld-1 gld-2 double mutants that , combined with either loss-of-function or gain-of-function glp-1 alleles , have morphologically similar gonads , filled with proliferating , undifferentiated germ cells ( Figure 2\\u2014figure supplement 1 ) ( Kadyk and Kimble , 1998; Hansen et al . , 2004 ) . We combined gld-1 ( q497 ) gld-2 ( q485 ) mutations with either the temperature-sensitive loss-of-function ( lf ) glp-1 allele ( e2144 ) , or the temperature-sensitive gain-of-function ( gf ) glp-1 allele ( ar202 ) ( Priess et al . , 1987; Pepper et al . , 2003 ) . Because GLD-1 and GLD-2 regulate gene expression at the posttranscriptional level only , we expected that transcriptionally regulated GLP-1Notch targets could be identified in this background . To analyze gene expression , gonads were dissected from animals grown at the restrictive temperature in two independent experiments , and transcripts were analyzed with tiling arrays ( GEO accession number GSE49395 ) . We identified around 100 transcripts that were differentially expressed between the gld-1 gld-2; glp-1 ( lf ) ( Notch OFF ) and gld-1 gld-2; glp-1 ( gf ) ( Notch ON ) gonads ( Figure 2A and Figure 2\\u2014figure supplement 2 , Figure 2\\u2014source data 1 ) . These changes were confirmed by quantitative RT-PCR ( RT-qPCR ) on selected transcripts ( Figure 2\\u2014figure supplement2A , Figure 2\\u2014source data 1 ) . Most differentially expressed transcripts were upregulated in the \\u2018Notch ON\\u2019 gonads , indicating a predominantly activating role of GLP-1Notch in germ cells . For simplicity , we will refer to the transcripts upregulated at least two fold in the Notch ON gonads as \\u2018Notch\\u2013activated\\u2019 . Some Notch-activated genes , such as sel-8/Mastermind , lst-1 , and epn-1/Epsin , have been implicated in Notch signaling in other cell types ( Doyle et al . , 2000; Yoo , 2004; Tian et al . , 2004; Singh et al . , 2011; Kershner et al . , 2014 ) . However , it remains possible that , rather than being direct GLP-1Notch targets , some of the Notch-activated genes were upregulated as an indirect consequence of increased GLP-1Notch signaling . 10 . 7554/eLife . 15477 . 008Figure 2 . GLP-1Notch and PRC2 regulate common targets and are functionally connected . ( A ) Notch-activated genes are biased for the sex chromosome linkage . Left: Changes in transcript abundance in the \\u2018Notch ON\\u2019 versus \\u2018OFF\\u2019 dissected gonads ( genotypes explained in Figure 2\\u2014figure supplement 1A\\u2013B ) were analyzed by microarrays . Transcripts upregulated at least 2-fold in the \\u2018Notch ON\\u2019 gonads are marked in red , those downregulated at least 2-fold in blue . Selected transcripts verified by RT-qPCR in Figure 2\\u2014figure supplement 2A are additionally circled in black . Right: 5426 genes can be considered expressed in the gonad , based on the bimodal distribution of expression values . Only 3% of those expressed genes are X-linked . In contrast , nearly half ( 46% ) of the expressed and Notch-activated transcripts are X-linked ( see Figure 2\\u2014figure supplement 2B for numbers ) . ( B ) GLP-1Notch and PRC2 interact genetically . Left: DAPI-stained gonads from animals of the indicated genotypes . The mes-2 ( bn11 ) M+Z- single mutant gonads have wild-type appearance at 20\\u00b0C . The glp-1 ( ar202 ) gain-of-function mutants have an almost wild-type appearance at this temperature , except for an extended proliferative zone in the gonad , referred to as 'distal tumor' . At the same temperature , mes-2 ( bn11 ) M+Z-; glp-1 ( ar202 ) double mutants developed germline tumors in 32/32 of the examined gonads . The insets show close-ups from the indicated gonadal regions: the distal-most regions contain undifferentiated , proliferative germ cells in all mutants ( a , c , e ) . However , while the single mutants contain oocytes with characteristically condensed chromosomes in the proximal gonads ( b , d ) , the proximal gonads of the double mutants harbor proliferative germ cells ( f ) . Scale bar = 30 \\u03bcm . Right: quantification of the phenotypes . 'Distal tumor' indicates the presence of an elongated distal proliferative zone ( approximately \\u00bd of the distal gonad arm ) . 'Extended' tumor indicates an extended distal tumor , few oocytes , and frequently also a proximal tumor . 'Fully tumorous' indicates the absence of all differentiated cell types except for sperm produced during larval development . ( C ) GLP-1Notch and PRC2 target the same genes on the X chromosomes . The plots correlate changes in gene expression in M+Z- mes-2 mutants with changes in gene expression changes in M+Z- mes-6 mutants . Results are shown separately for X-linked ( left ) and autosomal ( right ) transcripts . Notch-activated genes ( red in Figure 2A ) are marked in red . Lightly shaded areas indicate transcripts that are at least 2-fold upregulated . The overlap between transcripts upregulated by GLP-1Notch and transcripts upregulated by the loss of CePRC2 is highly significant , particularly for the X-linked genes . The significance of the correlation was measured by hypergeometric distribution; X-linked Notch-activated vs . mes-2 derepressed: p=1 . 31e-31; X-linked Notch-activated vs . mes-6 de-repressed: p=7 . 41e-25; autosomal Notch-activated vs . mes-2 derepressed: p=1 . 47e-22; autosomal Notch-activated vs . mes-6 de-repressed: p=1 . 8e-12 . DOI: http://dx . doi . org/10 . 7554/eLife . 15477 . 00810 . 7554/eLife . 15477 . 009Figure 2\\u2014source data 1 . Microarray results . ( A ) Microarray results of differentially expressed genes in Notch ON/OFF gonads . Figure 2\\u2014figure supplement 3 source data: Quantification of germlines with wild-type morphology versus germlines with tumors upon germline-autonomous RNAi against genetic interactors of PRC2 . More details can be found in the corresponding figure legends . DOI: http://dx . doi . org/10 . 7554/eLife . 15477 . 00910 . 7554/eLife . 15477 . 010Figure 2\\u2014figure supplement 1 . Examining transcriptional effects of GLP-1Notch signaling . ( A ) GLP-1Notch signaling counteracts the meiosis-promoting activity of several RNA binding proteins , of which GLD-1 and -2 are indicated . ( B ) Mutants that were used in this study: while the loss-of-function ( lf ) alleles of glp-1 cause the loss of germ cells , gain-of-function ( gf ) alleles result in a constitutive proliferation of germ cells . However , in the absence of the meiosis-promoting GLD-1 and GLD-2 proteins , germ cells continue to proliferate in the absence of GLP-1Notch activity . ( C ) Three representative gonads of Notch ON: gld-2 ( q497 ) gld-1 ( q485 ) ; glp-1 ( ar202 ) and Notch OFF: gld-2 ( a497 ) gld-1 ( q485 ) ; glp-1 ( e2144 ) animals are shown after dissection and DAPI staining . The central planes of the gonads were imaged . Nuclei were counted from those images using the CellCounter plugin with ImageJ . For each genetic background , germ cells in the entire gonad of 10 dissected gonads were counted . The quantification below revealed that the numbers of germ cells in both backgrounds are not changed . Error bars represent SD . Scale bar = 10 \\u03bcm . DOI: http://dx . doi . org/10 . 7554/eLife . 15477 . 01010 . 7554/eLife . 15477 . 011Figure 2\\u2014figure supplement 2 . Analysis of Notch-activated genes . ( A ) Changes in the abundance of several transcripts ( marked black in Figure 2A ) were verified independently from the genomic quantification by RT-qPCR . Colors correspond to fold-changes detected by microarrays ( green ) , by RT-qPCR on genetically identical gonads ( black ) , or by RT-qPCR on gonads with a different loss-of-function glp-1 allele , q175 ( white ) . *This experiment was performed only once . The error bars represent SEM . ( B ) Although the number of genes on the arrays is almost equally distributed between the different chromosomes , the expression of X-linked genes is largely silenced in the germline . The table displays the numbers and fractions of genes on the different chromosomes , and the distributions of the Notch-activated genes . The cutoff between 'not expressed' and 'expressed' genes was set according to the bimodal distribution of expression values in the Notch ON and OFF arrays . DOI: http://dx . doi . org/10 . 7554/eLife . 15477 . 01110 . 7554/eLife . 15477 . 012Figure 2\\u2014figure supplement 3 . The PRC2 component MES-6 and most enhancers/suppressors of glp-1 ( ar202 ) induced tumors appear to interact genetically with GLP-1Notch signaling in a germline-autonomous manner . Each bar indicates the relative proportion of germlines with wild-type morphology ( grey ) , germlines that contain eggs but also a proximal or distal tumor ( red ) , and germlines that are fully tumorous ( black ) . 'n' indicates the number of DAPI-stained gonads scored for each column . Empty vector 1\\u20133 represent three independent replicates of the empty vector control and demonstrate the robustness of the experiment . We observed that , for an unknown reason , the rrf-1 ( pk1417 ) ; glp-1 ( ar202 ) double mutants were less likely to produce tumors at the semi-permissive temperature of 20\\u00b0C . Nonetheless , the double mutant strain reacts to enhancers , suppressors , and depletion of PRC2 components in a similar manner as the glp-1 ( ar202 ) single mutant strain , with the exception of mbk-1 and B0416 . 5 RNAi . Depletion of mes-2 and mes-3 by RNAi was ineffective , since it did not enhance the tumourous phenotype in either of the two strains . DOI: http://dx . doi . org/10 . 7554/eLife . 15477 . 01210 . 7554/eLife . 15477 . 013Figure 2\\u2014figure supplement 4 . Global levels of H3K27me3 are unaffected by neither loss-of-function nor gain-of-function mutations in glp-1 . Stainings of wild-type , gain-of-function glp-1 ( ar202 ) , Notch ON ( gld-2 ( q497 ) gld-1 ( q485 ) ; glp-1 ( ar202 ) ) and Notch OFF ( gld-2 ( a497 ) gld-1 ( q485 ) ; glp-1 ( q175 ) ) gonads with antibodies against H3K27me3 performed on whole worms ( left panel , scale bars = 10 \\u03bcm ) or dissected gonads ( right panel , scale bars = 1 \\u03bcm ) . The H3K27me3 levels do not differ globally between the different mutant backgrounds . Asterisks indicate distal tips of gonads , dashed lines outline gonad . DOI: http://dx . doi . org/10 . 7554/eLife . 15477 . 013 To demonstrate that Notch-activated genes are functionally relevant for germ cell proliferation , we performed RNAi knockdown of Notch-activated genes ( n\\u00a0=\\u00a064 ) on animals carrying the gain-of-function glp-1 ( ar202 ) allele , and screened for enhancement or suppression of the tumorous gonad phenotype ( Supplementary file 1; for detailed experimental procedure see Materials and Methods ) . Knocking down some of the Notch-activated genes suppressed the tumorous phenotype , which agrees with predominantly proliferation-promoting role of GLP-1Notch . Interestingly , knocking down a smaller\\u00a0subset of the Notch-activated genes enhanced the tumor ( Supplementary file 1 ) , suggesting that some of the Notch-activated genes may counteract proliferation . While some of these genes may function autonomously in the germline , others could affect the germline indirectly from the soma . To test this , we RNAi-depleted selected candidates in the rrf-1 ( pk1417 ) mutant background , which is permissive for RNAi in the germline but deficient in RNAi in many ( but not all ) somatic tissues ( Kumsta and Hansen , 2012 ) . While depleting most candidates in the rrf-1 background had similar effects on the germline as in the wild type ( suggesting germline-autonomous function ) , in some cases the effects were abolished , suggesting that these genes function in the soma ( Figure 2\\u2014figure supplement 3 , Figure 2\\u2014source data 1 ) . Strikingly , we noticed that Notch-activated genes were enriched on the X-chromosome , the C . elegans sex chromosome . 45% of the Notch-activated genes were X-linked , which is four-fold more than expected by chance ( p=2 . 99e-14; Figure 2A and Figure 2\\u2014figure supplement 2B , Figure 2\\u2014source data 1 ) . When analyzing only genes with higher than baseline germline expression values , the disproportional X-linkage of Notch-activated genes was even more striking ( fifteen times more than expected by chance ( p=2 , 19e-38; Figure 2A and Figure 2\\u2014figure supplement 2B , Figure 2\\u2014source data 1 ) . In the C . elegans germline , X-linked genes are largely silenced by the C . elegans PRC2 ( Fong et al . , 2002 ) . Thus , the X chromosome bias among Notch-activated genes suggested a possible antagonistic relationship between GLP-1Notch and PRC2 . The C . elegans PRC2 consists of MES-2 , -3 , and -6 ( Bender et al . , 2004 ) and levels of the corresponding transcripts were essentially not altered by GLP-1Notch signaling ( mes-2: absolute fold change ( fc ) -1 . 3747; mes-3: fc 1 . 003; mes-6: fc 1 . 037 ) . To test for a functional relationship between GLP-1Notch and PRC2 , we examined genetic interactions between GLP-1 and PRC2 mutants . At 20\\u00b0C , both the temperature-sensitive gain-of-function glp-1 ( ar202 ) and the homozygous loss-of-function mes-2 ( bn11 ) mutants , derived from heterozygous mothers providing maternal MES-2 ( mes-2\\u2009M+Z- mutants ) , were viable and produced gonads with nearly wild-type appearance . The double mes-2\\u2009M+Z-; glp-1 ( ar202 ) mutants , however , displayed distal and proximal tumors at the same temperature ( Figure 2B; 32/32 examined gonads ) . PRC2 and GLP-1Notch thus interact functionally , and they appear to do so in a germ cell autonomous manner ( Figure 2\\u2014figure supplement 3 , Figure 2\\u2014source data 1 ) . Given the striking enrichment of Notch-activated genes on the X chromosome , and the genetic interaction between PRC2 and GLP-1Notch , we speculated that GLP-1Notch and PRC2 act on similar genes . To determine whether Notch-activated genes are also PRC2-repressed , we first determined putative PRC2 targets by expression analyses on isolated wild-type , M+Z- mes-2 or mes-6 mutant gonads ( GEO accession number GSE49395 ) . Consistent with the joint function of MES-2 and MES-6 in the PRC2 complex , a very similar set of genes was upregulated upon the loss of either protein ( Figure 2C; Figure 2\\u2014source data 1; henceforth \\u2018PRC2 repressed\\u2019 genes; also see ( Gaydos et al . , 2012 ) . Indeed , those PRC2-repressed genes overlapped strongly with Notch-activated genes , particularly those linked to the X chromosome . Nearly all of the X-linked Notch-activated genes were also derepressed upon the loss of PRC2 ( Figure 2C ) . This is consistent with the observed genetic interaction and suggests that increased GLP-1Notch signaling can induce expression of specific PRC2-repressed genes . This activation of the PRC2-repressed genes is not due to a global loss of the repressive tri-methylation of Histone H3 at lysine residue 27 ( H3K27me3 ) , since , upon examining gonads of different GLP-1Notch mutants , we observed no global loss of H3K27me3 in the germline ( Figure 2\\u2014figure supplement 4 ) . Germ cell conversion to neurons can be triggered not only by LIN-53 depletion but also by the depletion of PRC2 ( Patel et al . , 2012 ) . Potentially , the depletion of LIN-53 could facilitate reprograming by inhibiting PRC2 , since the depletion of LIN-53 results in a global loss of H3K27me3 in the germline ( Patel et al . , 2012 ) . Considering the antagonistic relation between GLP-1Notch and PRC2 on cell proliferation and gene regulation , we wondered whether GeCo triggered by the depletion of PRC2 would be sensitive to Notch signaling . Indeed , GeCo was strongly enhanced in PRC2-depleted ( mes-2 , mes-3 or mes-6 RNAi ) animals , when they also carried the gain-of-function glp-1 ( ar202 ) allele ( Figure 3A , Figure 3\\u2014source data 1 ) . Moreover , similar to the LIN-53\\u2013depleted gld-1 ( q497 ) gld-2 ( q485 ) ; glp-1 ( q175 ) gonads ( Figure 1D ) , the loss of GLP-1 effectively prevented GeCo in PRC2\\u2013depleted gld-1 ( q497 ) gld-2 ( q485 ) ; glp-1 ( q175 ) gonads ( Figure 3B , Figure 3\\u2014source data 1 ) . Together , these results suggest that GLP-1Notch stimulates GeCo in PRC2-compromised gonads . 10 . 7554/eLife . 15477 . 014Figure 3 . GLP-1Notch enhances reprograming upon the depletion of PRC2 . ( A ) Notch and PRC2 genetically interact in GeCo . Left: Fluorescent micrographs of glp-1 ( ar202 ) gain-of-function mutants expressing CHE-1\\u2013induced neuronal GFP . The animals were subjected to control RNAi or RNAi against PRC2 components ( MES-2 , 3 , and 6 ) , as indicated . Increased GLP-1Notch signaling enhanced the GeCo+ phenotype upon PRC2 depletion . Control RNAi ( mock ) for each genetic background did not result in any GeCo ( images not shown \\u2013 see quantification ) . Right: The corresponding quantifications . P-values were calculated using Student's t-test: p1=0 , 0006; p2<0 , 0001; p3=0 , 0536; p4=0 , 0001; p5=0 , 4035; p6=0 , 0003 . At least 200 animals were scored per condition . Error bars represent SEM . ( B ) GLP-1Notch is required for GeCo in PRC2-depleted animals independently from proliferation . Left: Fluorescent micrographs of adults expressing CHE-1\\u2013induced nGFP , with the genotypes indicated above the panels . The animals were subjected to RNAi as indicated on the left . The first column shows heterozygous , the other two homozygous animals carrying the loss of function alleles gld-1 ( q497 ) , gld-2 ( q485 ) and , in the central panels , glp-1 ( q175 ) . The animals were subjected to control RNAi or RNAi against PRC2 components ( MES-2 , 3 , and 6 ) . In the absence of GLP-1Notch , depletion of PRC2 components did not induce GeCo . Scale bars = 10 \\u03bcm . Right: The corresponding quantifications . P-values were calculated using Student's t-test: p1<0 , 0456; p2=0 , 0337; p3=0 , 0070; p4=0 , 0637; p5=0 , 0080; p6=0 , 1259 . At least 70 animals were scored per condition . Error bars represent SEM . DOI: http://dx . doi . org/10 . 7554/eLife . 15477 . 01410 . 7554/eLife . 15477 . 015Figure 3\\u2014source data 1 . Quantification of GeCo upon PRC2 depletion . ( A ) Quantification of GeCo+ phenotype upon RNAi against PRC2 subunits in glp-1 ( ar202 ) mutants . ( B ) Quantification of GeCo in different genetic backgrounds with highly proliferative germlines upon RNAi against PRC2 subunits . More details can be found in the corresponding figure legends . DOI: http://dx . doi . org/10 . 7554/eLife . 15477 . 015 To determine how GLP-1Notch might counteract PRC2 , we depleted candidate Notch-activated genes ( Supplementary file 1 ) , and examined GeCo efficiency ( Figure 4A , Figure 4\\u2014source data 1 ) . The strongest suppression of the GeCo+ and GeCo phenotype was observed upon the depletion of utx-1 , which also suppressed mes-3 RNAi-mediated GeCo+ in glp-1 ( ar202 ) gonads ( Figure 4\\u2014figure supplement 1 ) . Depletion of two other candidates , the uncharacterized C07G1 . 6 and the aldolase ortholog aldo-1 , also suppressed GeCo+ , albeit less efficiently ( Figure 4A ) . Because UTX-1 was suggested to effect gonadal development by functioning in the somatic gonad ( Vandamme et al . , 2012 ) , we re-examined GeCo efficiency upon utx-1 RNAi in the rrf-1 ( pk1417 ) background . Importantly , in rrf-1 mutants , RNAi is impaired in the somatic gonad , including the d In the C . elegans germline , GLP-1Notch signaling is essential for maintaining a pool of undifferentiated stem cells/progenitors . Here , we report an unexpected role of GLP-1Notch signaling in promoting cell fate reprograming . To understand this phenomenon , we identified genes activated upon increased GLP-1Notch signaling . While the identified genes include the physiological GLP-1Notch targets , sygl-1 and lst-1 , many other genes , including utx-1 , appear to be only weakly or not expressed in the distal-most region of the wild-type gonad , where both sygl-1 and lst-1 are induced by GLP-1Notch ( Kershner et al . , 2014 ) . Thus , the wild-type levels of GLP-1Notch signaling are either insufficient to induce expression of many potential target genes ( see below ) , or their expression is controlled by additional mechanisms , perhaps similarly to lip-1 mRNA , which , while induced by GLP-1Notch , is post-transcriptionally degraded in the distal-most gonad ( Hajnal and Berset , 2002;\\u00a0Lee et al . , 2006 ) . In addition to its major proliferation-promoting function , GLP-1Notch has been suggested to have a lesser role in promoting germ cell differentiation ( Hansen et al . , 2004 ) . Some of the identified Notch-activated genes appear to promote germ cell differentiation , potentially explaining the proposed differentiation-promoting function of GLP-1Notch . However , whether these genes are activated by GLP-1Notch and promote differentiation under physiological conditions remains to be determined . Many of the Notch-activated genes are repressed by PRC2 , suggesting that the expression of these genes is determined by the crosstalk between the extrinsic intercellular signaling pathway , Notch , and the intrinsic chromatin modifier PRC2 . Indeed , at least in the case of utx-1 , PRC2 appears to prevent its inappropriate expression in the distal-most gonad , which , nevertheless , can be overcome upon increased GLP-1Notch signaling . Our findings suggest that GLP-1Notch antagonizes PRC2 , at least in part , by stimulating expression of the H3K27 demethylase UTX-1 , which is essential for the enhancement of cellular reprograming . Previously , it was suggested that UTX-1 influences the germline by functioning in the somatic gonad ( Vandamme et al . , 2012 ) . However , by using the rrf-1 background , which displays defective RNAi in the somatic gonad , including the\\u00a0DTC ( Kumsta and Hansen , 2012 ) , we found that the reprograming-promoting role of UTX-1 is unlikely to depend on its function in the somatic gonad . Although we cannot fully exclude the possibility that the reprograming-facilitating role of UTX-1 depends on its expression in another somatic tissue ( such as the intestine , in which RNAi remains functional in the rrf-1 mutant; Kumsta and Hansen , 2012 ) , the germline expression of UTX-1 reported here suggests a germline-autonomous function . Consistent with this hypothesis , manipulating either PRC2 or GLP-1Notch affected the germline expression of utx-1 . Although additional factors , such as the uncharacterized C07G1 . 6 and the ortholog of the human aldolase A ( Kuwabara and O'Neil , 2001; Shaye and Greenwald , 2011 ) might contribute to reprograming , they appear to be less critical . In addition to UTX-1 , another H3K27/H3K9-demethylating enzyme , JMJD-1 . 2 ( Kleine-Kohlbrecher et al . , 2010 ) , is required for enhanced reprograming . Similar to UTX-1 , JMJD-1 . 2 is likely to be directly involved in regulating chromatin accessibility , since its depletion results in increased levels of the repressive H3K9me2 and H3K27me2 modifications ( Kleine-Kohlbrecher et al . , 2010 ) . The reprograming-promoting role of JMJD-1 . 2 might indicate that , besides further demethylation of H3K27me2 , perhaps also the removal of H3K9me2 facilitates GeCo . Future studies will shed light on this interesting question . UTX-1 orthologs in other species contribute to tissue-specific chromatin signatures , for example during myogenesis or in cardiac development ( Seenundun et al . , 2010; Lee et al . , 2012 ) , and have been implicated in germ-cell and somatic reprograming ( Mansour et al . , 2012 ) . Together with our data , these findings underscore the importance of UTX-1 in cellular reprograming . Here , we suggest that one way the activity of UTX-1 may be stimulated to promote reprograming is through its Notch signaling-dependent transcriptional activation . Interestingly , an antagonistic relationship between Notch and PRC2 has also\\u00a0been observed in T-cell leukemia ( Ntziachristos et al . , 2012 ) . A fascinating possibility is that a regulatory principle described here could help explain the etiology of this and perhaps other human diseases linked to a pathological increase in Notch signaling . Standard procedures were used to maintain animals . Worms ( RRID:WB_DL226 ) were grown at 25\\u00b0C unless stated otherwise . All heat-shock and temperature-sensitive strains were kept at 15\\u00b0C . The C . elegans lines used in this study are listed and described in detail in the Supplementary file 3A . The enhancer-suppressor screen on Notch targets was performed by feeding the animals with bacteria containing RNAi clones from the Ahringer and Vidal RNAi libraries . The clones not present in either of these libraries were cloned using primers as described in detail in the Supplementary file 3B . Experiments were performed at 15\\u00b0C , 20\\u00b0C or 25\\u00b0C using bleached embryos or overnight-synchronized L1 animals as stated in Supplementary file 1 . The percentage of adult animals with the germline tumor phenotype was scored . To test germ-cell autonomy , RNAi clones that induced significant suppression or enhancement in either setting were re-tested in a strain carrying the glp-1 ( ar202 ) ( RRID:WB_GC833 ) allele and , additionally the rrf-1 ( pk1417 ) ( RRID:WB_NL2098 ) mutation , which impairs RNAi primarily in the soma ( Sijen et al . , 2001 ) . For this test , gravid adults were picked to RNAi plates and allowed to lay eggs overnight at the semi-permissive temperature of 20\\u00b0C . Progeny was scored for enhancement or suppression of the germline tumorous phenotype at the young adult stage by DAPI staining of dissected gonads and scoring gonads as either wild-type , as containing a proximal or distal tumor but still some eggs , or as completely tumorous . Reprograming experiments were carried out as F1-RNAi . Worms were put on RNAi plates and the following F1 generation was screened for phenotypes . Used RNAi clones are described in the Supplementary file 3B . The genotype of the worms used for germ cell reprograming assays is BAT28 ( otIs305 ntIs1 , RRID:WB_OH9846 , details in Supplemental Materials and methods ) . Animals were synchronized by bleaching and eggs were put on NGM-agar containing E . coli OP50 ( RRID:WB_OP50 ) as a\\u00a0food source to grow at 15\\u00b0C until worms reached the L4-stage . At this stage 15\\u201320 worms were put per well of a 6-well plate , containing bacteria expressing dsRNA or carrying an empty RNAi vector , and grown at 15\\u00b0C until most of the F1 progeny reached the L3/L4 stage . The plates were heat-shocked at 37\\u00b0C for 30\\u2009min followed by an overnight incubation at 25\\u00b0C . Next day ( ~16\\u2009hrs post heat-shock ) the plates were screened for progeny showing ectopic GFP induction in the germline . To induce the Glp\\u00a0phenotype in glp-1 ( ar202 ) , the animals were shifted to room temperature 9\\u2009hrs before the heat-shock . For double RNAi , bacteria were grown as saturated culture . The OD600 was measured to ensure that the bacteria were mixed in an appropriate 1:1 ratio and subsequently seeded on RNAi 6-well plates . The RNAi screen was performed as described above . The p-values were calculated using Student's t-test . Hydroxyurea ( HU ) treatment was carried out as previously described ( Fox et al . , 2011; Patel et al . , 2012 ) . HU was added to seeded RNAi-plates at a final concentration of 250 \\u00b5M . L4 worms ( strain BAT32 , details in Supplementary file 3A ) grown on RNAi-plates were transferred to HU plates and incubated at room temperature for 5\\u2009hrs prior to heat-shock in order to induce CHE-1 expression . To test HU efficiency , control animals were treated with HU for 12\\u2009hrs with subsequent staining for DAPI and H3Ser10ph ( pH3 ) antibody ( Abcam #ab5176 ) . After overnight incubation , the worms were assessed for GFP induction in the germline as described above . To assess the efficiency of the HU treatment , the E . coli strain MG1693 ( E . coli stock center ) , with incorporated 5-ethynyl-20-deoxyuridine ( EdU ) , was used to feed L3/L4 worms . EdU in combination with DAPI staining was performed similar to the procedure described previously ( Ito and McGhee , 1987 ) and according to the manufacturer's instructions ( Invitrogen , Europe ) of the EdU labeling kit . The Click-iT EdU reaction buffer ( Invitrogen , Europe ) was mixed with Alexa Fluor azide ( \\u2018click\\u2019 reaction ) dye to detect cells that were replicating DNA . Staining was performed by freeze cracking worms after fixation with 2% formaldehyde . Parental animals were raised at 15\\u00b0C and shifted to 25\\u00b0C for egg laying . Offspring was dissected after the L4-adult transition . Fifty gonads per analyzed genotype in triplicates were dissected in M9 containing levamisole . RNA was isolated using the PicoPure RNA Isolation Kit ( Applied Biosystems ) according to the manufacturer\\u2019s recommendations . Total RNA ( 200 ng for the Notch ON/OFF experiments or 100 ng for the mes/wild-type experiments ) was amplified using the Affymetrix GeneChip WT Amplified Double Stranded cDNA Synthesis Kit according to the manufacturer\\u2019s instructions . The Affymetrix GeneChip WT Double-Stranded cDNA Terminal Labeling Kit was used for the fragmentation and labeling of 7 . 5\\u2009\\u03bcg cDNA . The labeled material was loaded onto Affymetrix GeneChip C . elegans Tiling 1 . 0R Arrays and hybridized at 65\\u00baC for 16\\u2009hrs . The arrays were washed in an Affymetrix Fluidics station with the protocol FS450_0001 and scanned in an Affymetrix GeneChip Scanner 3000 with autoloader using Affymetrix GCC Scan Control v . 3 . 0 . 0 . 1214 software . All tiling arrays were processed in R ( 32 , 33 ) using Bioconductor ( 34 ) and the packages tilingArray and preprocessCore . The arrays were RMA background corrected and log2 transformed on the oligo level using the following command: expr <- log2 ( rma . background . correct ( exprs ( readCel2eSet ( filenames , rotated = TRUE ) ) ) ) . We mapped the oligos from the tiling array ( bpmap file from www . affymetrix . com ) to the C . elegans genome assembly ce6 ( www . genome . ucsc . edu ) using bowtie ( Langmead et al . , 2009 ) , allowing no error and unique mapping position . Expressions for individual transcripts were calculated by intersecting the genomic positions of the oligos with transcript annotation ( WormBase WS190 ) and averaging the intensity of the respective oligos . For the mes-4/wt experiment , no quantile normalization was performed as the distributions for the mes-4 mutant and the wt differed substantially . In the case of the Notch ON/OFF dataset quantile normalization was performed on the level of transcripts . The p-values were calculated in R with the hypergeometric distribution function 'phyper' . RNA was isolated as described above . cDNA was synthesized with oligo dT primer using the ImProm II Reverse Transcription System from Promega according to the manufacturer\\u2019s instructions . cDNA was used for qPCR with the Absolute QPCR SYBR Green ROX Mix ( AbGene ) on an ABI PRISM 7700 system ( Applied Biosystems ) . qPCR reactions were performed as described previously ( Biedermann et al . , 2009 ) . At least one primer in each pair is specific for an exon-exon junction . Sequences of the used primers are described in detail in the Supplementary file 3C . Mouse RNA was added before RNA isolation and reverse transcription , allowing for normalization to cyt-c and thereby correcting for variations in RNA isolation and reverse transcription . Standard curves for quantification were generated from a serial dilution of input cDNA for each primer pair . The amount of target present in each replicate was derived from the standard curve; an average was calculated for each of the triplicates . To compare total mRNA levels , the qPCR results were normalized to mouse cyt-c and to the C . elegans tubulin gene tbb-2 , and fold enrichments were calculated . RNA in situ hybridization was performed and analyzed as previously described ( Biedermann et al . , 2009 ) . Description of the primer pairs to generate probes from cDNA can be found in the Supplementary file 3C . Images were captured with a Zeiss AxioImager Z1 microscope , equipped with a Zeiss AxioCam MRc camera . Images were taken in linear mode of the Axiovision software ( Zeiss ) and processed with Adobe Photoshop CS4 in an identical manner . ChIP was performed as described ( Askjaer et al . , 2014 ) . In brief , worms ( strains OP591 , RRID:WB_OP591 , and BAT890 ) were washed with M9 buffer and frozen as \\u2018worm popcorn\\u2019 in liquid nitrogen prior to pulverization with a mortar and pestle . The powder was dissolved in 10 volumes of 1 , 1% formaldehyde in PBS + 1\\u00a0mM PMSF and incubated 10\\u2009mins with gentle rocking at room temperature with subsequent quenching for 5\\u2009mins at room temperature by adding 2 , 5 molar glycine ( final concentration 125\\u2009mM ) . After centrifugation with 4 . 000\\u2009g at 4\\u00b0C the pellet was washed with ice-cold PBS+1\\u00a0mM PMSF and resuspended in 50\\u2009mM FA buffer ( 50\\u2009mM HEPES/KOH pH 7 , 5; 1\\u00a0mM EDTA; 1% Triton-X 100; 0 , 1% sodium deoxycholate; 150\\u2009mM NaCl ) + 1% sarkosyl + protease-inhibitors . Samples were sonicated twice using the Biorupter ( 15 times 30 s ON , 30 s OFF ) on high settings at 4\\u00b0C followed by spinning at 13 . 000\\u2009g , 15\\u2009min , 4\\u00b0C . The supernatant corresponding to 4 mg of protein measured by Bradford assay was used for ChIPs . Samples were incubated with 50 \\u00b5l of FLAG or HA antibodies coupled to \\u00b5MACS microbeads ( Milteny ) ( all blocked with 5% milk in FA-buffer ) . After incubating 1\\u2009hr at 4\\u00b0C , the beads where washed in \\u00b5MACS matrix columns in a magnetic rack with FA buffer containing 1\\u00a0M and 0 . 5\\u2009M NaCl and subsequent wash with TE and TEL buffer ( 0 , 25\\u2009M LiCl; 1% Sodium deoxycholate; 1\\u2009mM EDTA; 10\\u2009mM Tris pH 8 , 0 ) . Bound material was eluted with 65\\u00b0C pre-warmed 125 \\u00b5l ChIP elution buffer ( 1% SDS , 250\\u2009mM NaCl , 10\\u2009mM Tris pH 8 . 0 , 1\\u2009mM EDTA ) and fixation was reversed using 2 \\u00b5l of 10 mg/ml Proteinase K at at 50\\u00b0C for 1\\u2009hr followed by 65\\u00b0C incubation overnight . DNA was purified using the Quiaquick PCR purification kit in a final volume of 40 \\u00b5l and 1 \\u00b5l was used for qPCR . Negative controls were used to assess the quality of the ChIP and fold enrichment of the target genes: lysates ( N2 worms ) which do not express the recombinant target protein with specific antibody ( anti-FLAG coupled to \\u00b5MACS beads ) and lysates of worms expressing the recombinant target protein with unspecific antibody ( anti-HA coupled to \\u00b5MACS beads ) controls , respectively . Primer for qPCRs was designed using Primer3Plus ( Untergasser et al . , 2007 ) with the following settings: amplified region min . 100 bp \\u2013 max . 200 bp; GC content: 50\\u201360%; min . primer length: 18 nt \\u2013 max . length 24 nt; melting temperature: min . 58\\u00b0C \\u2013 max . 63\\u00b0C; max . ; 3' self complementary allowance set to 1; max . allowed length of a mononucleotide repeat ( max . poly-x ) : 3 . Sequences of the used primers are listed in in the Supplementary file 3C . The qPCRs were run on CFX96 Touch Real-Time PCR Detection System from BioRad using the Maxima SYBR Green/ROX qPCR Master Mix ( 2X ) . The data analysis was performed by calculating the \\u0394\\u0394Ct-values . Differences were assessed using Student's t-test . Worms were transferred to a slide and fixed by adding 10\\u201320 \\u00b5l 95% ethanol and letting evaporate the ethanol . The ethanol fixation was repeated 2 more times before adding the DAPI solution in microscopy mounting media ( vectashield from Vector or similar ) . The samples were sealed with a coverslip and nail polish before microscopy . Fluorescent micrographs were recorded with Zeiss AxioImager Z1 microscope and the SensicamII camera ( PCO ) and the micromanager software was used to capture Z-stack images with 0 . 5 \\u00b5m slice steps . Images subject to direct comparison were taken at identical exposure times . Counting of germ cells within the range from the DTC to the turn of gonadal arms of glp-1 ( ar202 ) ; hsp::che-1; gcy-5::gfp animals treated with either mock or lin-53 RNAi was performed using the Z-stacks . Micromanager was used to control the Z-stack levels and the ImageJ plugin for cell counting for scoring the number of germ cells . Germ cell counts in gonads of Notch ON: gld-2 ( q497 ) gld-1 ( q485 ) ; glp-1 ( ar202 ) and Notch OFF: gld-2 ( a497 ) gld-1 ( q485 ) ; glp-1 ( e2144 ) ( Figure 1\\u2014figure supplement 2 ) and germline tumor phenotype in glp-1 ( ar202 ) and glp-1 ( ar202 ) ; rrf-1 ( pk1417 ) were scored after dissection , formaldehyde fixation and DAPI staining . For Notch ON and Notch OFF mutants , the central plane of the gonads was imaged and germ cells in the entire dissected gonad were counted using the CellCounter plugin with ImageJ . For each of the two strains , germ cells in the entire gonad of 15 dissected gonads were counted . Antibody stainings of intact worms were performed using a freeze-crack procedure as described ( Duerr , 2006 ) . In brief , after washing , worms were resuspended in 0 . 025% glutaraldehyde , and frozen between two frost-resistant glass slides on dry ice . Separating the glass slides while frozen creates additional cracks in the cuticle . Acetone/methanol or 4% paraformaldehyde in 0 . 1\\u00a0M phosphate buffer for 1\\u2009hr on ice fixation was used . Worms were washed off the slides in PBS , blocked with 0 . 2% gelatin + 0 . 25% Triton in PBS , and stained . Primary antibodies were diluted in PBS with 0 . 1% gelatin and 0 . 25% Triton and fixed worms were incubated 4\\u2009hrs - overnight at 4\\u00b0C . After PBS washes secondary antibody was applied for 3\\u2009hrs . After PBS washes worms were mounted with DAPI-containing mounting medium ( Dianova , #CR-38448 ) on glass slides . The primary antibodies used were anti-H3K27me3 ( 1:400; gift from Dr . Hiroshi Kimura ) ; anti-HA ( 1:100 , Roche #12CA5; acetone fixation ) ; anti-H3Ser10ph ( 1:400 , Abcam , #ab14955; acetone fixation ) . Secondary antibodies were Alexa Fluor dyes applied at 1:1000 dilution . Stainings for H3K27me3 on dissected gonads were performed using anti-H3K27me3 from Millipore ( catalogue number 07\\u2013449 , Lot 1959680; courtesy of Jan Padeken , Gasser laboratory ) on dissected gonads . The\\u00a0adult animals were dissected in M9 containing levamisole , fixed with 2% paraformaldehyde in PBS on poly-lysine coated slides , snap-frozen on dry ice , freeze-cracked , incubated for 5 mins in ice-cold DMF at \\u221220\\u00b0 , washed for 5\\u2009mins in PBS 0 . 1% Tween-20 at room temperature , blocked for 20\\u2009mins in PBS 0 . 1% Tween-20 + 5% BSA and incubated with the primary antibody over night at 4\\u00b0C . Secondary antibodies ( Alexa 488 , goat anti rabbit , 1:500 ) were applied for 2\\u00a0hrs at room temperature . Slides were then washed three times for 5\\u2009mins in PBS 0 . 1% Tween-20 at room temperature and mounted with Vectashield mounting medium containing DAPI . The transcriptional reporter gene putx-1::gfp-h2b ( rrrSi185 ) was constructed from the 1302 bp putative promoter region of the gene utx-1 ( human UTX ( Ubiquitously transcribed TPR on X ) homolog - 1 ) fused to sequences encoding for GFP-H2B and the ubiquitously expressed tbb-2 3\\u2019UTR using the Gateway Reporter Cloning System ( Merritt et al . , 2008 ) . The reporter gene putx-1::gfp-h2b ( rrrSi185 and rrrSi281 ) was constructed with the following primers: putx-attB4: GGGGACAACTTTGTATAGAAAAGTTGGGATTTTATCTTCATCGGACCTG putx-attB1 : GGGGACTGCTTTTTTGTACAAACTTGTGGCGGTGTGAGAAGCGATAC The full-length functional transgene putx-1::flag-gfp-utx-1::utx-1 3\\u2019UTR ( rrrSi189 ) was constructed with the following primers: utx-1+3UTR+attB2 L ggggacagctttcttgtacaaagtggACGACGAATCAGAACCTCTGCCGGAGGAGCGTCATgtaag utx-1+3UTR+attB3 R ggggacaactttgtataataaagttgaatgcggatactgccttctc The functional UTX-1 transgene putx-1::flag-gfp-tev::utx-1::utx-1 3\\u2019utr ( rrrSi189 ) contains the same promoter as the transcriptional reporter , the full-length utx-1 genomic sequence as well as the endogenous 3\\u2019UTR , and was equipped with N-terminal GFP , FLAG , and TEV tags . Transgenic animals were produced\\u00a0as\\u00a0single-copy integrants\\u00a0using the MosSCI direct insertion protocol ( Fr\\u00f8kjaer-Jensen et al . , 2008 ) . The rrrSi189 transgene is functional , as it rescues the utx-1 mutant ( ok3553 allele ) . For GFP quantification , gonads were dissected from live animals in M9 buffer containing levamisole and mounted to glass microscopy slides ( Fr\\u00f8kjaer-Jensen et al . , 2008 ) . Fluorescent micrographs were recorded with Zeiss AxioImager Z1 microscope and a Zeiss Axioncam MRm REV 2 CCD camera was used to capture images . Fluorescence intensities were quantified using ImageJ . GFP intensities were normalized to the picture background and corrected with the average autofluorescences measured in wild type ( N2 ) gonads at the corresponding temperatures . Images subject to direct comparison were taken at identical exposure times and were processed with Adobe Photoshop CS4 in an identical manner . The numbers of analyzed gonads were as follows: n = 44 for wild-type reporter; n = 36 for glp-1 ( gf ts ) ; n = 55 for wild-type reporter on control RNAi; n = 48 for mes-2 ( RNAi ) ; n = 15 for mes-3 ( RNAi ) ; n = 29 for mes-6 ( RNAi ) , and n = 20 for LAG-1 binding sites deleted reporter . Alleles used were glp-1 ( ar202 ) and mes-2 ( bn11 ) . Worms were grown at the semi-permissive temperature of 20\\u00b0C and gonads were dissected and DAPI-stained shortly after the L4-young adult transition . The experiment was performed twice . In a first round , a low number of gonads were examined to identify whether the double mutant had a phenotype and to define phenotypic categories to score . Based on the observation of a clear and penetrant phenotype , gonads were scored in a second round according to the categories defined in the first round .\"]" # @param {type:"string"}

In [ ]:
#@title Method 1: Normal RAG, designed to establish a baseline summary using a simple Retrieval-Augmented Generation (RAG) approach
# import os
# import time
# import csv
# from groq import Groq
# from langchain_core.documents import Document
# from langchain.text_splitter import RecursiveCharacterTextSplitter
# from qdrant_client import QdrantClient
# from qdrant_client.http import models
# from sentence_transformers import SentenceTransformer

# Placeholder for the original content (15,000+ words)
# content = "Your 15000+ word content goes here..."

# Set the Groq API key as an environment variable
os.environ["GROQ_API_KEY"] = ""
GROQ_API_KEY = os.environ.get("GROQ_API_KEY")

if not GROQ_API_KEY:
    raise ValueError("Groq API key not found. Please set the GROQ_API_KEY environment variable.")

# Initialize Groq client
client = Groq(api_key=GROQ_API_KEY)

### Utility Functions

def get_completion(prompt, temperature=0.7, model="mistral-saba-24b"):
    """Generate a completion using the Groq API."""
    try:
        chat_completion = client.chat.completions.create(
            messages=[{"role": "user", "content": prompt}],
            model=model,
            temperature=temperature
        )
        return chat_completion.choices[0].message.content
    except Exception as e:
        print(f"Error during get_completion: {e}")
        return None

def retrieve(qdrant_client, model, query, k=6):
    """Retrieve relevant context from Qdrant."""
    query_embedding = model.encode([query])[0]
    results = qdrant_client.query_points(
        collection_name="content_collection",
        query=query_embedding.tolist(),
        limit=k
    ).points
    return [result.payload["text"] for result in results]

### Main Execution

if __name__ == "__main__":
    start_time = time.time()

    # Initialize SentenceTransformer model
    model = SentenceTransformer("nomic-ai/nomic-embed-text-v1", trust_remote_code=True)

    # Split the content into chunks
    document = Document(page_content=content)
    splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=100)
    splits = splitter.split_documents([document])

    # Initialize Qdrant client (in-memory)
    qdrant_client = QdrantClient(":memory:")

    # Create Qdrant collection
    if not qdrant_client.collection_exists("content_collection"):
        qdrant_client.create_collection(
            collection_name="content_collection",
            vectors_config=models.VectorParams(size=model.get_sentence_embedding_dimension(), distance=models.Distance.COSINE),
        )

    # Generate embeddings and upload to Qdrant
    embeddings = model.encode([split.page_content for split in splits])
    qdrant_client.upload_points(
        collection_name="content_collection",
        points=[
            models.PointStruct(id=i, vector=embedding.tolist(), payload={"text": split.page_content})
            for i, (split, embedding) in enumerate(zip(splits, embeddings))
        ]
    )

    # Retrieve relevant chunks
    retrieval_query = "Summarize the content based on the main ideas and key points."
    retrieved_context = retrieve(qdrant_client, model, retrieval_query, k=6)

    # Build summarization prompt
    context_text = "\n".join(retrieved_context)
    summarization_prompt = f"""
    Summarize the following content:

    **Content:**
    {context_text}

    Create a comprehensive summary exceeding 280 words with a clear structure (introduction, body, conclusion).
    Maintain a formal tone, ensure natural flow, and include all key elements for coherence.
    """

    # Generate five summaries
    print("Performing summarization (five iterations)...")
    summaries = []
    for i in range(5):
        print(f"Generating summary {i + 1}...")
        summary = get_completion(summarization_prompt)
        if summary:
            summaries.append(summary)
            print(f"Summary {i + 1} generated successfully.")
        else:
            summaries.append("Summary generation failed.")
            print(f"Summary {i + 1} failed.")
        time.sleep(10)  # Additional delay between summaries to respect rate limits

    # Save summaries to CSV
    with open("summaries_method1-1.csv", "w", newline="", encoding="utf-8") as file:
        writer = csv.writer(file)
        headers = [f"Summary_{i+1}" for i in range(5)]
        writer.writerow(headers)
        writer.writerow(summaries)

    print("\nGenerated Summaries saved to 'summaries_method1-1.csv'.")
    print(f"\nTotal execution time: {(time.time() - start_time):.2f} seconds")